In [1]:
import mackinac
import cobra
import pandas as pd
import json
import os

/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# load ID's for each organisms genome
id_table = pd.read_table('../data/patric_genomes.csv',sep=',',\
                        dtype='str')
species_to_id = dict(zip(id_table["Species_name"],id_table["PATRIC_GENOME_ID"]))

In [3]:
species_to_id

{'Achromobacter piechaudii': '742159.3',
 'Achromobacter xylosoxidans': '85698.28',
 'Aeromonas salmonicida': '382245.13',
 'Bacillus megaterium': '545693.3',
 'Bacillus pumilus': '1408.93',
 'Bacillus subtilis subsp. Spizizenii': '655816.3',
 'Brachybacterium faecium': '446465.5',
 'Brevundimonas diminuta': '751586.3',
 'Chryseobacterium gleum': '525257.3',
 'Corynebacterium efficiens': '196164.6',
 'Corynebacterium glutamicum': '196627.13',
 'Enterococcus faecalis': '226185.9',
 'Flavobacterium johnsoniae': '376686.10',
 'Gordonia bronchialis': '526226.15',
 'Haemophilus influenzae': '71421.8',
 'Haemophilus parasuis': '557723.8',
 'Kytococcus sedentarius': '478801.5',
 'Listeria monocytogenes': '169963.11',
 'Listeria seeligeri': '683837.3',
 'Neisseria flavescens': '546264.5',
 'Neisseria gonorrhoeae': '242231.10',
 'Neisseria meningitidis': '122586.8',
 'Neisseria mucosa': '435832.3',
 'Pseudomonas mendocina': '399739.8',
 'Pseudomonas putida': '160488.4',
 'Pseudomonas stutzeri':

In [4]:
mackinac.get_token('gregmedlock_seed')


patric password: ········


'gregmedlock_seed@patricbrc.org'

In [5]:
# grab and save a universal model to be used later for gapfilling. This is a public template available in Mike Mundy's workspace.
# The template says "gramneg", but there is no difference between the g+ and g- templates other than biomass composition,
# which will not be used during gapfilling (the GENREs will already have their own biomass function).
gramneg = mackinac.create_universal_model('/mmundy/public/modelsupport/templates/MicrobialNegativeResolved.modeltemplate')
cobra.io.save_json_model(gramneg,'../data/universal_mundy.json')

In [6]:
# save id's and both names in dictionary
name_to_recon_info = {}
name_to_gapfill_solution = {}
for species in species_to_id.keys():
    # Check for an existing GENRE
    if species+'.json' not in os.listdir('../data/modelseed_models'):
        species_id = species_to_id[species]

        # reconstruct model; function returns a dictionary with reconstruction info, NOT the model
        print("Reconstructing GENRE for " + species)
        recon_info = mackinac.create_patric_model(species_id,species)
        name_to_recon_info[species] = recon_info
        # Get the reactions contained in the gapfill solution. This is on complete media
        name_to_gapfill_solution[species] = mackinac.get_patric_gapfill_solutions(species)[0]
        # convert to a cobra model
        model = mackinac.create_cobra_model_from_patric_model(species)
        # Save model in json format using patric_id
        cobra.io.save_json_model(model, '../data/modelseed_models/'+species+'.json')

# save conversion dict for id:original_name:SEED_name mapping
with open('../data/patric_recon_info.json','w') as jsonfile:
    json.dump(name_to_recon_info,jsonfile)
    
# save the gapfill solutions
with open('../data/patric_gapfill_solutions.json','w') as jsonfile:
    json.dump(name_to_gapfill_solution,jsonfile)
    

Reconstructing GENRE for Neisseria mucosa
Reconstructing GENRE for Salmonella enterica subsp. enterica serovar Typhi
Reconstructing GENRE for Corynebacterium glutamicum
Reconstructing GENRE for Flavobacterium johnsoniae
Reconstructing GENRE for Aeromonas salmonicida
Reconstructing GENRE for Brevundimonas diminuta
Reconstructing GENRE for Gordonia bronchialis
Reconstructing GENRE for Pseudomonas stutzeri
